In [ ]:
WINDOW = 200
RADIUS = 1

In [ ]:
import numpy as np
import pandas as pd

def generate_data(df):
    target_indices = []
    timestamps = []
    
    for i in range(WINDOW, len(df)):
        target_indices.append(i)
        timestamps.append(df.iloc[i].name)
        
    return target_indices, timestamps

In [ ]:
import numpy as np

def create_image(df):
    size = len(df)
    array = np.zeros((size, size, 3), dtype=np.uint8)

    epsilon = 1e-10
    log_high = np.log(df['High'].values + epsilon)
    log_low = np.log(df['Low'].values + epsilon)
    log_open = np.log(df['Open'].values + epsilon)
    log_close = np.log(df['Close'].values + epsilon)
    
    # NaNを含む時刻のマスクを作成
    nan_mask = (np.isnan(log_high) | np.isnan(log_low) | 
                np.isnan(log_open) | np.isnan(log_close))
    
    global_min = np.nanmin(log_low)
    global_max = np.nanmax(log_high)
    center = (global_min + global_max) / 2

    price_min = center - RADIUS
    price_max = center + RADIUS

    # 価格を画像のy座標にマッピング（0が上端で高値、size-1が下端で安値）
    def price_to_y(price):
        # price_maxが上端(0)、price_minが下端(size-1)
        y = (price_max - price) / (price_max - price_min) * (size - 1)
        # NaNの場合は-1を返す（後で無効化するため）
        y = np.where(np.isnan(price), -1, y)
        return np.clip(y, -1, size - 1).astype(int)
    
    # 各時点の価格をy座標に変換
    y_high = price_to_y(log_high)
    y_low = price_to_y(log_low)
    y_open = price_to_y(log_open)
    y_close = price_to_y(log_close)
    
    # x座標（時間軸）のインデックス配列
    x_indices = np.arange(size)
    
    # チャンネル1: HighからLowまでの範囲を255で塗りつぶす
    for i in range(size):
        if not nan_mask[i]:
            # y_high[i]からy_low[i]まで塗りつぶす（y_high <= y_low なので注意）
            array[y_high[i]:y_low[i]+1, i, 0] = 255
    
    # チャンネル2: Open < Closeの場合、OpenからCloseまでを255で塗りつぶす（陽線）
    bullish = log_open < log_close  # 上昇（陽線）
    for i in range(size):
        if bullish[i] and not nan_mask[i]:
            # CloseがOpenより高い（yは小さい）
            y_min = min(y_close[i], y_open[i])  # 上端
            y_max = max(y_close[i], y_open[i])  # 下端
            array[y_min:y_max+1, i, 1] = 255
    
    # チャンネル3: Close < Openの場合、CloseからOpenまでを255で塗りつぶす（陰線）
    bearish = log_close < log_open  # 下降（陰線）
    for i in range(size):
        if bearish[i] and not nan_mask[i]:
            # OpenがCloseより高い（yは小さい）
            y_min = min(y_open[i], y_close[i])  # 上端
            y_max = max(y_open[i], y_close[i])  # 下端
            array[y_min:y_max+1, i, 2] = 255
    
    return array

In [ ]:
import joblib

file_path = 'history/6758.T.joblib'
history_data = joblib.load(file_path)

ticker = history_data['ticker']
history_df = history_data['history_df']

# データの作成
target_indices, timestamps = generate_data(history_df)

In [ ]:
from PIL import Image
import os

output_dir = 'preview_images'
os.makedirs(output_dir, exist_ok=True)

for timestamp, target_indice in zip(timestamps, target_indices):
    img = create_image(history_df.iloc[target_indice - WINDOW: target_indice])
    img = Image.fromarray(img)
    save_path = f"{output_dir}/{ticker}_{timestamp.isoformat()}.png"
    img.save(save_path)